In [ ]:
!pip install transformers==2.10.0

     |████████████████████████████████| 665kB 2.5MB/s 
     |████████████████████████████████| 3.8MB 6.3MB/s 
     |████████████████████████████████| 1.1MB 29.9MB/s 
     |████████████████████████████████| 890kB 20.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=3c2cc90d2fd4bb5a62bdd69fc4380eb5d2e8d4b8b72bfe673624e3edec3ba2e8
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
df = pd.read_csv('./drive/My Drive/Reviews.csv')


In [ ]:
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [ ]:
df.loc[df.Score == 5].sample(5)[['Summary', 'Text']]

,Summary,Text
378653,Got me on the tea bandwagon,When in Boulder this summer I took a tour of t...
31688,It's the Cats' Meow,This is one of the best products that we have ...
546841,good dog food,switched over to this because of the new puppy...
165476,great price for a great product,"I love Eight O'clock coffee, and I love grindi..."
528832,My favorite,I'll be honest only the home made house french...


In [ ]:
df.loc[df.Score == 1].sample(5)[['Summary', 'Text']]

,Summary,Text
136831,kind of gross,I love green tea and maybe I'm just not used t...
428196,These bubbles scare the kitties - indoor and o...,I got a bottle of these bubbles for my brother...
499382,Kio is dying..,"After reading the reviews, I searched the web ..."
70311,Garbage! False Description!,All you can taste is artificial sweetener. It ...
38279,Duped,Because they showed six barbells for the price...


In [ ]:
import numpy as np
sentences = df.Text.values
labels = df.Score.values - 1

#sentences = df.Summary.values

#sentences = sentences[:1000]
#labels = (labels[:1000]).astype(int)

idx = np.where(labels==2)
labels = np.delete(labels, idx)
sentences = np.delete(sentences, idx)
#labels = np.array([l for l in labels if l in {0,1,3,4}])
labels = (labels>2).astype(int)
print(labels[0:10])

[1 0 1 0 1 1 1 1 1 1]


In [ ]:
from transformers import RobertaTokenizer
max_len = 0

tokenizer = RobertaTokenizer.from_pretrained('roberta-base', do_lower_case=True)

In [ ]:
import torch
input_ids = []
attention_masks = []

# For every sentence...
for sent in sentences:
    encoded_dict = tokenizer.encode_plus(
                        sent, 
                        add_special_tokens = True, 
                        max_length = 128,           
                        pad_to_max_length = True,
                        return_attention_mask = True,  
                        return_tensors = 'pt'   
                   )  
    input_ids.append(encoded_dict['input_ids'])
    
    attention_masks.append(encoded_dict['attention_mask'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
print('phase score:', labels[2])
labels = torch.tensor(labels)

print('Original: ', sentences[2])
print('Token IDs:', input_ids[2])
print('phase score:', labels[2])

phase score: 1
Original:  This is a confection that has been around a few centuries.  It is a light, pillowy citrus gelatin with nuts - in this case Filberts. And it is cut into tiny squares and then liberally coated with powdered sugar.  And it is a tiny mouthful of heaven.  Not too chewy, and very flavorful.  I highly recommend this yummy treat.  If you are familiar with the story of C.S. Lewis' "The Lion, The Witch, and The Wardrobe" - this is the treat that seduces Edmund into selling out his Brother and Sisters to the Witch.
Token IDs: tensor([    0,    42,    16,    10,  7856, 20970,    14,    34,    57,   198,
           10,   367, 11505,     4,  1437,    24,    16,    10,  1109,     6,
        26410,   219, 28460, 44496,    19, 15092,   111,    11,    42,   403,
        14242,  1943,  1872,     4,     8,    24,    16,   847,    88,  5262,
        32657,     8,   172, 25232,  2368, 31444,    19, 39143,  4696,     4,
         1437,     8,    24,    16,    10,  5262,  6085,  2650,

In [ ]:
from torch.utils.data import TensorDataset, random_split

dataset = TensorDataset(input_ids, attention_masks, labels)
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

473,232 training samples
52,582 validation samples


In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
batch_size = 32

train_dataloader = DataLoader(
            train_dataset,  
            sampler = RandomSampler(train_dataset), 
            batch_size = batch_size 
        )

validation_dataloader = DataLoader(
            val_dataset, 
            sampler = SequentialSampler(val_dataset), 
            batch_size = batch_size 
        )

In [ ]:
from transformers import RobertaForSequenceClassification, AdamW, BertConfig

model = RobertaForSequenceClassification.from_pretrained(
    "roberta-base", 
    num_labels = 2,  
    output_attentions = False, 
    output_hidden_states = False, 
)

model.cuda()

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm

In [ ]:
optimizer = AdamW(model.parameters(),
                  lr = 4e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

from transformers import get_linear_schedule_with_warmup

epochs = 4

total_steps = len(train_dataloader) * epochs

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [ ]:
import numpy as np

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
#torch.cuda.empty_cache()
#PATH = '/content/drive/My Drive/data'
#torch.save(model.state_dict(),PATH + '//'+ str(50))

In [ ]:
import random
import numpy as np

seed_val = 22
epochs = 4
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
train_loss_list = []
dev_acc_list = []
PATH = '/content/drive/My Drive/data'
iter = 0 
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):
    

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()
    total_train_loss = 0

    model.train()

    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        if step % 2000 == 0 and not step == 0:
            torch.save(model, "/content/drive/My Drive/results/model_saved_" + str(epoch_i) + "_"+ str(step) + ".pt")
            
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        model.zero_grad()       

        outputs = model(b_input_ids, 
                    token_type_ids=None, 
                    attention_mask=b_input_mask, 
                    labels=b_labels)
        loss = outputs[0]

        total_train_loss += loss.item()
        # Perform a backward pass to calculate the gradients.
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
    # Calculate the average loss over the training data.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Store the loss value for plotting the learning curve.
    train_loss_list.append(avg_train_loss)
    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))


    print("")
    print("Running Validation...")

    t0 = time.time()
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        with torch.no_grad():        
            (loss, logits) = model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        total_eval_accuracy += flat_accuracy(logits, label_ids)

    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    avg_val_loss = total_eval_loss / len(validation_dataloader)

    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,+
            'Valid. Accur.': avg_val_accuracy,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 4 ========
Training...
  Batch    40  of  14,789.    Elapsed: 0:00:16.
  Batch    80  of  14,789.    Elapsed: 0:00:31.
  Batch   120  of  14,789.    Elapsed: 0:00:46.
  Batch   160  of  14,789.    Elapsed: 0:01:01.
  Batch   200  of  14,789.    Elapsed: 0:01:16.
  Batch   240  of  14,789.    Elapsed: 0:01:31.
  Batch   280  of  14,789.    Elapsed: 0:01:46.
  Batch   320  of  14,789.    Elapsed: 0:02:01.
  Batch   360  of  14,789.    Elapsed: 0:02:16.
  Batch   400  of  14,789.    Elapsed: 0:02:31.
  Batch   440  of  14,789.    Elapsed: 0:02:47.
  Batch   480  of  14,789.    Elapsed: 0:03:02.
  Batch   520  of  14,789.    Elapsed: 0:03:17.
  Batch   560  of  14,789.    Elapsed: 0:03:32.
  Batch   600  of  14,789.    Elapsed: 0:03:47.
  Batch   640  of  14,789.    Elapsed: 0:04:02.
  Batch   680  of  14,789.    Elapsed: 0:04:17.
  Batch   720  of  14,789.    Elapsed: 0:04:32.
  Batch   760  of  14,789.    Elapsed: 0:04:47.
  Batch   800  of  14,789.    Elapsed: 0:05:0

In [ ]:
model

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm

In [ ]:
"/content/drive/My Drive/results"

'/content/drive/My Drive/results'

In [ ]:
torch.save(model,"/content/drive/My Drive/results/roberta_final.pt")